In [1]:
from keras.models import Sequential
from keras.layers import Input,Conv2D,Lambda,LeakyReLU,Activation,Concatenate,MaxPooling2D,BatchNormalization,Flatten,Dense,Dropout,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from matplotlib import pyplot as plt
from keras.applications.mobilenet import MobileNet
#import tensorflow as tf
from keras.models import Model
from tensorflow.keras import metrics
import numpy as np #矩阵库
import cv2         #图像处理库
import matplotlib #绘图库
import pandas as pd      #数据处理库
import os
import sys
import random
from datetime import datetime
from subprocess import check_output

ImageValData={}
def loadTrainData(trainDir): #读取训练图像数据及处理
    global ImageValData
    
    images=os.listdir(trainDir)
    print("验证集图片共有"+str((len(images)))+"张")
    
    for imageFileName in images:
        imageFullPath=os.path.join(trainDir,imageFileName)
        img=cv2.imread(imageFullPath)
        arr=img
        #print(arr.shape)
          #提取各颜色空间并生成LGI颜色空间
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
        Blue=arr[:,:,0]
        Green=arr[:,:,1]
        Red=arr[:,:,2]
        #print(Green.shape)
        Green=clahe.apply(Green)
        #print(Green.shape)
        Lab=cv2.cvtColor(arr,cv2.COLOR_BGR2Lab)
        Light=Lab[:,:,0]
        Light=clahe.apply(Light)
        #print(Light.shape)
        I1=cv2.cvtColor(arr,cv2.COLOR_BGR2GRAY)
        #print(I1.shape)
        I1=clahe.apply(I1)
        arr=cv2.merge([Green,Light,I1])
        dim0=arr.shape[0]#高度
        dim1=arr.shape[1]#宽度
        dim2=arr.shape[2]#深度
        #调整图片尺寸统一
        if (dim0<128 or dim1<128 or dim2<3):
            print("图片错误"+str(arr.shape))
        arr=cv2.resize(arr,(128,128))
        dim0=arr.shape[0]#新高度
        dim1=arr.shape[1]#新宽度
        dim2=arr.shape[2]#新深度
        #确定修改完成
        if (dim0!=128 or dim1!=128 or dim2!=3):
            print("图片错误"+str(arr.shape))
        #print(arr)
        arr = np.array(arr, dtype="float") / 255.0
        imageFileName = imageFileName.replace('.jpg','')
        ImageValData [str(imageFileName)] = np.array(arr)
    print('完成')
    return

print("读取数据"+str(datetime.now()))
sys.stdout.flush()
loadTrainData("a/b. Testing Set")


Using TensorFlow backend.


读取数据2020-04-29 02:09:40.891537
验证集图片共有103张
完成


In [2]:
def loadValLabels():#读取标签
    raw_df=pd.read_csv('b/b.csv',sep=',',usecols=[0,1])
    row_count=raw_df.shape[0]#数据总量
    col_count=raw_df.shape[1]#列数
    print("共有"+str(row_count)+"数据\n"+"拥有"+str(col_count)+"特征")
    header_list=list(raw_df.columns)#检测特征含义
    print("特征分为:"+str(header_list))
    return raw_df
Val_label=loadValLabels()
Val_keepImages =list(ImageValData.keys())
print("完成")

共有103数据
拥有2特征
特征分为:['Image name', 'Retinopathy grade']
完成


In [3]:
val_image_arr=[]
val_data_arr=[]

for index, row in Val_label.iterrows():
    key = str(row[0])
    if key in ImageValData:
        val_image_arr.append(key)
        val_data_arr.append(np.array(ImageValData [key]))
Val_label2 = pd.DataFrame({'Image name': val_image_arr, 'data': val_data_arr})

Val_label = pd.merge(Val_label2, Val_label, left_on='Image name', right_on='Image name', how='outer')
label_header_list2 = list(Val_label.columns)

print(label_header_list2)
print('测试集有', str(len(Val_label))+"张图片")
print("完成")

['Image name', 'data', 'Retinopathy grade']
测试集有 103张图片
完成


In [4]:
valset = Val_label.reset_index(drop=True)
print(valset.head())
print("完成")

  Image name                                               data  \
0  IDRiD_001  [[[0.011764705882352941, 0.011764705882352941,...   
1  IDRiD_002  [[[0.03137254901960784, 0.0196078431372549, 0....   
2  IDRiD_003  [[[0.011764705882352941, 0.0196078431372549, 0...   
3  IDRiD_004  [[[0.011764705882352941, 0.011764705882352941,...   
4  IDRiD_005  [[[0.011764705882352941, 0.011764705882352941,...   

   Retinopathy grade  
0                  4  
1                  4  
2                  4  
3                  4  
4                  4  
完成


In [5]:
valX = valset['data']
valY = valset['Retinopathy grade']
name = valset['Image name']
print('验证集为', valX.shape[0])

训练集为 103


In [6]:
from keras.models import load_model
model = load_model('MobileNetstep1.h5')
model2=load_model('MobileNetstep2.h5')
model3=load_model('MobileNetstep3.h5')
T=0
TP=0
TN=0
FN=0
FP=0
I00=0
I01=0
I02=0
I03=0
I04=0
I10=0
I11=0
I12=0
I13=0
I14=0
I20=0
I21=0
I22=0
I23=0
I24=0
I30=0
I31=0
I32=0
I33=0
I34=0
I30=0
I31=0
I32=0
I33=0
I34=0
pre=np.zeros(103)
for i in range(103):
    arr = np.expand_dims(valX[i], axis=0)
    preds = model.predict(arr)
    if preds[0,0]>preds[0,1]:
        pre[i]=0    
    else:
        preds2 = model2.predict(arr)
        if preds2[0,0]<preds2[0,1]:
            pre[i]=4 
        else:
            preds3=model3.predict(arr)
            if (preds3[0,0]>preds3[0,1])&(preds3[0,0]>preds3[0,2]):
                pre[i]=1
            elif (preds3[0,1]>preds3[0,0])&(preds3[0,1]>preds3[0,2]):
                pre[i]=2
            elif (preds3[0,2]>preds3[0,0])&(preds3[0,2]>preds3[0,1]):
                pre[i]=3

for i in range(103):
    if (pre[i]==0)&(valY[i]!=0):
        FN=FN+1
    elif (pre[i]==0)&(valY[i]==0):
        TN=TN+1
    elif (pre[i]!=0)&(valY[i]==0):
        FP=FP+1
    elif (pre[i]!=0)&(valY[i]!=0):
        TP=TP+1

        
for i in range(103):
    if pre[i]==valY[i]:
        T=T+1
        #print(name[i])

print('TN:'+str(TN))
print('TP:'+str(TP))
print('FN:'+str(FN))
print('FP:'+str(FP))
P=TP+FN
print('准确率为'+str(T/103))
print('召回率为'+str(TP/P))
    

W0429 02:11:52.648527  2616 deprecation_wrapper.py:119] From D:\Users\fyh980928\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0429 02:11:53.585687  2616 deprecation_wrapper.py:119] From D:\Users\fyh980928\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0429 02:11:53.700683  2616 deprecation_wrapper.py:119] From D:\Users\fyh980928\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0429 02:11:53.702678  2616 deprecation_wrapper.py:119] From D:\Users\fyh980928\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_d

TN:32
TP:63
FN:6
FP:2
准确率为0.8155339805825242
召回率为0.9130434782608695


In [8]:
I00=0
I01=0
I02=0
I03=0
I04=0
I10=0
I11=0
I12=0
I13=0
I14=0
I20=0
I21=0
I22=0
I23=0
I24=0
I30=0
I31=0
I32=0
I33=0
I34=0
I40=0
I41=0
I42=0
I43=0
I44=0
for i in range(103):
    if (pre[i]==0)&(valY[i]==0):
        I00=I00+1
    elif (pre[i]==0)&(valY[i]==1):
        I01=I01+1
    elif (pre[i]==0)&(valY[i]==2):
        I02=I02+1
    elif (pre[i]==0)&(valY[i]==3):
        I03=I03+1
    elif (pre[i]==0)&(valY[i]==4):
        I04=I04+1
for i in range(103):
    if (pre[i]==1)&(valY[i]==0):
        I10=I10+1
    elif (pre[i]==1)&(valY[i]==1):
        I11=I11+1
    elif (pre[i]==1)&(valY[i]==2):
        I12=I12+1
    elif (pre[i]==1)&(valY[i]==3):
        I13=I13+1
    elif (pre[i]==1)&(valY[i]==4):
        I14=I14+1
for i in range(103):
    if (pre[i]==2)&(valY[i]==0):
        I20=I20+1
    elif (pre[i]==2)&(valY[i]==1):
        I21=I21+1
    elif (pre[i]==2)&(valY[i]==2):
        I22=I22+1
    elif (pre[i]==2)&(valY[i]==3):
        I23=I23+1
    elif (pre[i]==2)&(valY[i]==4):
        I24=I24+1
for i in range(103):
    if (pre[i]==2)&(valY[i]==0):
        I20=I20+1
    elif (pre[i]==2)&(valY[i]==1):
        I21=I21+1
    elif (pre[i]==2)&(valY[i]==2):
        I22=I22+1
    elif (pre[i]==2)&(valY[i]==3):
        I23=I23+1
    elif (pre[i]==2)&(valY[i]==4):
        I24=I24+1
for i in range(103):
    if (pre[i]==4)&(valY[i]==0):
        I40=I40+1
    elif (pre[i]==4)&(valY[i]==1):
        I41=I41+1
    elif (pre[i]==4)&(valY[i]==2):
        I42=I42+1
    elif (pre[i]==4)&(valY[i]==3):
        I43=I43+1
    elif (pre[i]==4)&(valY[i]==4):
        I44=I44+1
print(I00)
print(I01)
print(I02)
print(I03)
print(I04)
print(I10)
print(I11)
print(I12)
print(I13)
print(I14)
print(I20)
print(I21)
print(I22)
print(I23)
print(I24)
print(I30)
print(I31)
print(I32)
print(I33)
print(I34)
print(I40)
print(I41)
print(I42)
print(I43)
print(I44)

32
1
4
0
1
0
1
0
0
0
4
6
56
8
4
0
0
0
0
0
0
0
0
1
9
